In [1]:
from unified_planning.io.pddl_reader import PDDLReader
from unified_planning.model import Problem
from unified_planning.shortcuts import *
from bitblast.helpers import *
from pathlib import Path
import os
import numpy as np
from sympy import *

In [2]:
reader = PDDLReader()

problem = reader.parse_problem(Path("/home/lubonassi/code/BitBlast/test/pddl/counters/domain.pddl"), Path("/home/lubonassi/code/BitBlast/test/pddl/counters/instance_4.pddl"))

In [3]:
problem

problem name = instance_4

types = [counter]

fluents = [
  real value[c=counter]
  real max_int
]

actions = [
  action increment(counter c) {
    preconditions = [
      ((value(c) + 1) <= max_int)
    ]
    effects = [
      value(c) += 1
    ]
  }
  action decrement(counter c) {
    preconditions = [
      (1 <= value(c))
    ]
    effects = [
      value(c) -= 1
    ]
  }
]

objects = [
  counter: [c0, c1, c2, c3]
]

initial fluents default = [
]

initial values = [
  max_int := 8
  value(c0) := 0
  value(c1) := 0
  value(c2) := 0
  value(c3) := 0
]

goals = [
  (((value(c0) + 1) <= value(c1)) and ((value(c1) + 1) <= value(c2)) and ((value(c2) + 1) <= value(c3)))
]


In [4]:
simplifier = Simplifier(environment=get_environment(), problem=problem)
checker = LinearChecker(environment=get_environment(), problem=problem)

In [12]:
# Transform all effects into INCREASE
def set_normalized_effect(act: InstantaneousAction, eff: Effect):
    if eff.is_increase():
        act.add_increase_effect(condition=eff.condition, value=eff.value, fluent=eff.fluent)
    elif eff.is_decrease():
        act.add_increase_effect(condition=eff.condition, value=simplifier.simplify(-eff.value), fluent=eff.fluent)
    else:
        # TODO: handle increase and decrease effects expressed as assignments
        raise NotImplementedError("Effect type not supported")
    

def normalize_action(act: InstantaneousAction) -> InstantaneousAction:
    normalized_action = InstantaneousAction(act.name)
    
    for pre in act.preconditions:
        if is_numeric_condition(pre):
            expression = deep_simplify(Minus(lhs(pre), rhs(pre)))
        if is_le(pre):
            normalized_action.add_precondition(LE(expression, 0))
        elif is_lt(pre):
            normalized_action.add_precondition(LT(expression, 0))
        elif is_equals(pre):
            normalized_action.add_precondition(Equals(expression, 0))
        else:
            normalized_action.add_precondition(pre)

    
    for eff in act.effects:
        set_normalized_effect(normalized_action, eff)
    return normalized_action
        

In [13]:
normalized_actions = [normalize_action(act) for act in problem.actions]
normalized_actions[0]

action increment {
    preconditions = [
      ((1 + value(c) + (-1 * max_int)) <= 0)
    ]
    effects = [
      value(c) += 1
    ]
  }

In [7]:
def lhs(formula: FNode):
    return formula.args[0]

def rhs(formula: FNode):
    return formula.args[1]

In [8]:
def get_substitutions(fluents):
    '''
    Before converting an expression to a sympy expression, we need to substitute all the
    fluents with new variables var0, var1 ecc...
    This is because sympy interprets fluent(a, b) as a function, not as a variable.
    '''
    new_vars_dict = {}
    index = 0
    for fluent in fluents:
        new_fl = Fluent(f'var{index}', RealType())
        new_vars_dict[fluent] = new_fl()
        index += 1
    return new_vars_dict

extractor = FreeVarsExtractor()
linear_checker = LinearChecker()
nnf_transformer = Nnf(get_environment())
substituter = Substituter(get_environment())


